In [1]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
#Stochastic Petri Net Example 2
# Petri net modeled as queueing network
# Note: This is a queueing network representation of Petri net behavior
model = Network('model')

In [3]:
#Petri net with 4 places and 3 transitions
num_places = 4
num_transitions = 3 #Places represented as queues/delays
places = []
for i in range(num_places):
    if i == 0:
        place = Delay(model, f'Place{i}')  #Source place
    else:
        place = Queue(model, f'Place{i}', SchedStrategy.INF)  #Infinite server
    places.append(place) #Transitions represented as service processes
jobclass = ClosedClass(model, 'Tokens', 5, places[0], 3)

In [4]:
#Transition firing rates (service rates)
for i, place in enumerate(places):
    firing_rate = 1.0 + i * 0.2
    place.setService(jobclass, Exp(firing_rate))

In [5]:
#Petri net structure (simplified as routing)
P = model.initRoutingMatrix() #Cyclic structure representing token flow
for i in range(num_places - 1):
    P.set(jobclass, jobclass, places[i], places[i+1], 1.0)
    #Close the loop
    P.set(jobclass, jobclass, places[-1], places[0], 1.0)
model.link(P)

In [6]:
# Solve Petri net representation
solver = SolverCTMC(model)
avgTable = solver.getAvgTable()

  Station JobClass    QLen    Util   RespT  ResidT   ArvR   Tput
0  Place0   Tokens  1.5760  1.5760  1.0000  1.0000  1.576  1.576
1  Place1   Tokens  1.3133  1.3133  0.8333  0.8333  1.576  1.576
2  Place2   Tokens  1.1257  1.1257  0.7143  0.7143  1.576  1.576
3  Place3   Tokens  0.9850  0.9850  0.6250  0.6250  1.576  1.576
